In [2]:
## .env (Environment file)
## github secrets

MESO_API_KEY = ""

In [3]:
# Download a PDF ✅
# Process that PDF ✅
# Chunk the PDF into smaller parts ✅
# Convert those chunks into embeddings ✅
# Store those embeddings ✅

In [43]:
# !pip3 install pymupdf
# !pip3 install openai
# !pip3 install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 30.3 MB/s eta 0:00:00:00:010:01m

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
!wget https://www.tnb.com.my/assets/annual_report/TNB_IAR_2023.pdf

In [60]:
import fitz
import requests
import json
import numpy as np
from tqdm import tqdm
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
doc = fitz.open("/workspaces/RAG-Devtalk/TNB_IAR_2023.pdf")

In [18]:
for page in doc:
    print(page.get_text())
    break

75
75
I n t e g r a t e d  A n n u a l  R e p o r t  2 0 2 3
P O W E R I N G  G R O W T H 
C A T A L Y S I N G  G R E E N



In [11]:
from custom_chunk import process_document

In [12]:
chunks = process_document("/workspaces/RAG-Devtalk/TNB_IAR_2023.pdf", chunk_size=2048, overlap_size=128)

In [20]:
chunks[0]

('75 75 Integrated Annual Report 2023 POWERING GROWTH CATALYSING GREEN Our annual report cover celebrates TNB’s remarkable 75-year journey, depicted through intricate line drawings narrating two compelling storylines encapsulating evolution and growth. In the upper segment, the illustration eloquently traces Malaysia’s societal advancement, symbolising the transition from quaint rural landscapes to bustling urban metropolises. This narrative mirrors TNB’s trajectory, evolving from a traditional power provider to a pioneer championing sustainable energy solutions. It reflects the unwavering commitment to the Energy Transition Plan, with the ultimate aim of achieving Net Zero by 2050. As we celebrate our 75th anniversary, the cover serves as a testament to our enduring mission: Powering Growth, Catalysing Green. It showcases TNB’s leadership in fostering sustainable growth, spearheading the transition towards a greener future. The bottom illustration delves into the individual’s social a

In [21]:
url = "https://llm-router.nous.mesolitica.com/embeddings"

In [22]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {MESO_API_KEY}"
}

In [36]:
## Homework
### Add the PDF file name
### Apply batch processing

data_list = []

for chunk in tqdm(chunks):
    text, page = chunk
    # print(text)
    # print(page)

    payload = {
        "input": text,
        "model": "base"
    }


    response = requests.post(url, json=payload, headers=headers)

    json_response = response.json()

    # print(json_response)
    # print(json_response.keys())

    embedding = json_response["data"][0]["embedding"]

    embedding_array = np.array(embedding)

    data_input = {
        "text": text,
        "page": page,
        "embedding": embedding_array
    }

    data_list.append(data_input)

100%|██████████| 858/858 [02:00<00:00,  7.13it/s]


In [33]:
print(len(embedding))
print(embedding)

1024
[-0.0869140625, 0.007781982421875, -0.0169677734375, -0.014892578125, -0.0279541015625, -0.0142822265625, -0.0169677734375, -0.0155029296875, 0.00921630859375, 0.0634765625, 0.01953125, 0.0478515625, -0.018798828125, 0.01904296875, -0.046142578125, 0.0113525390625, 0.07275390625, 0.041748046875, -0.0211181640625, 0.0032958984375, -0.0211181640625, 0.006439208984375, 0.0194091796875, 0.005706787109375, -0.0001697540283203125, 0.058349609375, -0.0179443359375, -0.033447265625, -0.0213623046875, 0.03173828125, -0.024658203125, -0.017333984375, -0.017333984375, 0.00390625, -0.0174560546875, -0.043701171875, 0.033935546875, -0.044921875, -0.0361328125, -0.0223388671875, 0.040771484375, 0.0108642578125, 0.00762939453125, -0.0244140625, 0.060302734375, -0.046142578125, -0.04248046875, -0.0184326171875, -0.019287109375, 0.03271484375, -0.01068115234375, -0.0244140625, 0.0032501220703125, -0.0181884765625, 0.0255126953125, 0.0208740234375, -0.07470703125, 0.01031494140625, -0.032470703125,

In [38]:
data_list[0]

{'text': '75 75 Integrated Annual Report 2023 POWERING GROWTH CATALYSING GREEN Our annual report cover celebrates TNB’s remarkable 75-year journey, depicted through intricate line drawings narrating two compelling storylines encapsulating evolution and growth. In the upper segment, the illustration eloquently traces Malaysia’s societal advancement, symbolising the transition from quaint rural landscapes to bustling urban metropolises. This narrative mirrors TNB’s trajectory, evolving from a traditional power provider to a pioneer championing sustainable energy solutions. It reflects the unwavering commitment to the Energy Transition Plan, with the ultimate aim of achieving Net Zero by 2050. As we celebrate our 75th anniversary, the cover serves as a testament to our enduring mission: Powering Growth, Catalysing Green. It showcases TNB’s leadership in fostering sustainable growth, spearheading the transition towards a greener future. The bottom illustration delves into the individual’s 

In [40]:
df = pd.DataFrame(data_list)

In [41]:
df

,text,page,embedding
0,75 75 Integrated Annual Report 2023 POWERING G...,"[1, 2]","[-0.0869140625, 0.007781982421875, -0.01696777..."
1,"aspirations for a Better, Brighter Future. Fro...","[2, 3]","[-0.06494140625, 0.0137939453125, -0.010437011..."
2,Corporate Structure 22 Our International Footp...,[3],"[-0.0654296875, -0.00138092041015625, -0.02465..."
3,Board Leadership & Effectiveness 147 Effective...,"[3, 4]","[-0.024169921875, 0.008544921875, -0.010803222..."
4,52 FROM OUR LEADERSHIP 02 NAVIGATION ICONS Thi...,[4],"[-0.0213623046875, 0.0294189453125, 0.00273132..."
...,...,...,...
853,Proxy 2 NRIC No./PassportNo.: or failing him/h...,[383],"[-0.052734375, -0.0223388671875, 0.00534057617..."
854,payment of the following Non-Executive Directo...,[383],"[-0.05126953125, -0.05859375, -0.023193359375,..."
855,Director from the date of his appointment unti...,"[383, 384]","[-0.051025390625, -0.0233154296875, -0.0280761..."
856,"members of the Company, in accordance with Sec...",[384],"[-0.0186767578125, -0.0034332275390625, -0.019..."


In [44]:
df.to_parquet("data.parquet", engine="pyarrow")

In [45]:
# RAG
# Embeddings
# Input from user
# Convert input into embedding
# Map input_embedding to the list of embeddings
# Return back the relevant documents

In [46]:
input_str = "Apakah faktor yang meningkatkan keuntungan TNB?"

In [47]:
payload = {
    "input": input_str,
    "model": "base"
}

In [61]:
response = requests.post(url, json=payload, headers=headers)

json_response = response.json()

embedding = json_response["data"][0]["embedding"]

embedding_input = np.array(embedding)

In [62]:
embedding_input

array([-0.02233887,  0.00357056, -0.04052734, ..., -0.04345703,
        0.02600098, -0.05981445])

In [64]:
list_of_embeddings = df["embedding"].tolist()

In [66]:
cosine_similarities = cosine_similarity([embedding_input], list_of_embeddings).flatten()

In [ ]:
cosine_similarities

In [68]:
top_n = 10
top_indexes = cosine_similarities.argsort()[-top_n:][::-1]

In [ ]:
# Negative Embeddings
# top_indexes = cosine_similarities.argsort()[:10]

In [70]:
most_similar_items = [data_list[i] for i in top_indexes]

In [88]:
most_similar_items[0]

{'text': 'TNB are resolute in supporting the growth of the nation by creating further value for all our stakeholders by leveraging new opportunities and realise the nation’s sustainability ambitions for a better and brighter future. The increase in electricity demand directly contributed to TNB’s top-line growth of 4.3% YOY in 2023, with revenue growing to RM53.07 billion. However, we recorded lower EBITDA and profit after tax (PAT) of RM18.6 billion and RM2.6 billion respectively, mainly due to the negative fuel margin. Our liquidity and cashflow position have improved significantly in 2023 as we recorded a 56% reduction in receivables, mainly through consistent cost recovery of Imbalance Cost Pass-Through (ICPT) as well as efforts made by TNB to improve our collection rate. Operational excellence remains a cornerstone of our strategy, andwe strive to ensure our infrastructure assets remain robust and available to serve our customers. We recorded a Transmission System Minutes of 0.48 

In [73]:
import os
from openai import OpenAI

os.environ['OPENAI_API_KEY'] = MESO_API_KEY

client = OpenAI(
    base_url="https://llm-router.nous.mesolitica.com",
)

In [76]:
input_list = []

for item in most_similar_items:
    input_list.append(item["text"])

In [79]:
full_text = " ".join(input_list)

In [82]:
full_text

'TNB are resolute in supporting the growth of the nation by creating further value for all our stakeholders by leveraging new opportunities and realise the nation’s sustainability ambitions for a better and brighter future. The increase in electricity demand directly contributed to TNB’s top-line growth of 4.3% YOY in 2023, with revenue growing to RM53.07 billion. However, we recorded lower EBITDA and profit after tax (PAT) of RM18.6 billion and RM2.6 billion respectively, mainly due to the negative fuel margin. Our liquidity and cashflow position have improved significantly in 2023 as we recorded a 56% reduction in receivables, mainly through consistent cost recovery of Imbalance Cost Pass-Through (ICPT) as well as efforts made by TNB to improve our collection rate. Operational excellence remains a cornerstone of our strategy, andwe strive to ensure our infrastructure assets remain robust and available to serve our customers. We recorded a Transmission System Minutes of 0.48 minutes, 

In [83]:
prompt = f"Diberi data {full_text} \n\nUser: {input_str}"

In [84]:
prompt

'Diberi data TNB are resolute in supporting the growth of the nation by creating further value for all our stakeholders by leveraging new opportunities and realise the nation’s sustainability ambitions for a better and brighter future. The increase in electricity demand directly contributed to TNB’s top-line growth of 4.3% YOY in 2023, with revenue growing to RM53.07 billion. However, we recorded lower EBITDA and profit after tax (PAT) of RM18.6 billion and RM2.6 billion respectively, mainly due to the negative fuel margin. Our liquidity and cashflow position have improved significantly in 2023 as we recorded a 56% reduction in receivables, mainly through consistent cost recovery of Imbalance Cost Pass-Through (ICPT) as well as efforts made by TNB to improve our collection rate. Operational excellence remains a cornerstone of our strategy, andwe strive to ensure our infrastructure assets remain robust and available to serve our customers. We recorded a Transmission System Minutes of 0.

In [85]:
response = client.chat.completions.create(
    model="mallam-small",
    temperature=0.1,
    max_tokens=1024,
        messages=[
        {
            'role': 'system',
            'content': 'awak adalah chatbot yang dicipta untuk membantu manusia.'
        },
        {
            "role": "user",
            "content": prompt
        }
    ],
)

In [86]:
print(response.choices[0].message.content)

Faktor yang meningkatkan keuntungan TNB ialah:

1. Peningkatan jualan elektrik: Jualan elektrik yang lebih tinggi menyumbang kepada peningkatan keuntungan TNB.

2. Kecekapan operasi: Kecekapan dalam operasi TNB membantu mengurangkan kos dan meningkatkan keuntungan.

3. Pelaburan dalam teknologi: Pelaburan dalam teknologi baharu dan inovatif membantu TNB meningkatkan kecekapan operasi dan menawarkan perkhidmatan yang lebih baik kepada pelanggan.

4. Pengurusan risiko yang baik: TNB mengambil langkah untuk menguruskan risiko dengan baik, seperti risiko pasaran dan risiko kawal selia.

5. Pengurusan rantaian nilai yang cekap: TNB memastikan pengurusan rantaian nilai yang cekap dalam semua aktiviti perniagaannya.

6. Pengurusan modal insan yang baik: TNB mempunyai tenaga kerja yang berkualiti dan terlatih, yang membantu meningkatkan prestasi dan keuntungan syarikat.

7. Pengurusan risiko yang baik: TNB mengambil langkah untuk menguruskan risiko dengan baik, seperti risiko pasaran dan risik

In [89]:
response = client.chat.completions.create(
    model="mallam-small",
    temperature=0.1,
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Given the text, 'I love cream cheese sandwich with olives' to Malay and make the response in JSON with the key 'translated' and 'original'"
        }
    ],
)

In [90]:
response.choices[0].message.content

'{"translated": "Saya suka sandwic keju krim dengan buah zaitun", "original": "I love cream cheese sandwich with olives"}'

In [91]:
recipe_api = ""

In [ ]:
chunks[0]

('75 75 Integrated Annual Report 2023 POWERING GROWTH CATALYSING GREEN Our annual report cover celebrates TNB’s remarkable 75-year journey, depicted through intricate line drawings narrating two compelling storylines encapsulating evolution and growth. In the upper segment, the illustration eloquently traces Malaysia’s societal advancement, symbolising the transition from quaint rural landscapes to bustling urban metropolises. This narrative mirrors TNB’s trajectory, evolving from a traditional power provider to a pioneer championing sustainable energy solutions. It reflects the unwavering commitment to the Energy Transition Plan, with the ultimate aim of achieving Net Zero by 2050. As we celebrate our 75th anniversary, the cover serves as a testament to our enduring mission: Powering Growth, Catalysing Green. It showcases TNB’s leadership in fostering sustainable growth, spearheading the transition towards a greener future. The bottom illustration delves into the individual’s social a

In [ ]:
url = "https://llm-router.nous.mesolitica.com/embeddings"

In [ ]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {MESO_API_KEY}"
}

In [ ]:
## Homework
### Add the PDF file name
### Apply batch processing

data_list = []

for chunk in tqdm(chunks):
    text, page = chunk
    # print(text)
    # print(page)

    payload = {
        "input": text,
        "model": "base"
    }


    response = requests.post(url, json=payload, headers=headers)

    json_response = response.json()

    # print(json_response)
    # print(json_response.keys())

    embedding = json_response["data"][0]["embedding"]

    embedding_array = np.array(embedding)

    data_input = {
        "text": text,
        "page": page,
        "embedding": embedding_array
    }

    data_list.append(data_input)

100%|██████████| 858/858 [02:00<00:00,  7.13it/s]


In [ ]:
print(len(embedding))
print(embedding)

1024
[-0.0869140625, 0.007781982421875, -0.0169677734375, -0.014892578125, -0.0279541015625, -0.0142822265625, -0.0169677734375, -0.0155029296875, 0.00921630859375, 0.0634765625, 0.01953125, 0.0478515625, -0.018798828125, 0.01904296875, -0.046142578125, 0.0113525390625, 0.07275390625, 0.041748046875, -0.0211181640625, 0.0032958984375, -0.0211181640625, 0.006439208984375, 0.0194091796875, 0.005706787109375, -0.0001697540283203125, 0.058349609375, -0.0179443359375, -0.033447265625, -0.0213623046875, 0.03173828125, -0.024658203125, -0.017333984375, -0.017333984375, 0.00390625, -0.0174560546875, -0.043701171875, 0.033935546875, -0.044921875, -0.0361328125, -0.0223388671875, 0.040771484375, 0.0108642578125, 0.00762939453125, -0.0244140625, 0.060302734375, -0.046142578125, -0.04248046875, -0.0184326171875, -0.019287109375, 0.03271484375, -0.01068115234375, -0.0244140625, 0.0032501220703125, -0.0181884765625, 0.0255126953125, 0.0208740234375, -0.07470703125, 0.01031494140625, -0.032470703125,

In [ ]:
data_list[0]

{'text': '75 75 Integrated Annual Report 2023 POWERING GROWTH CATALYSING GREEN Our annual report cover celebrates TNB’s remarkable 75-year journey, depicted through intricate line drawings narrating two compelling storylines encapsulating evolution and growth. In the upper segment, the illustration eloquently traces Malaysia’s societal advancement, symbolising the transition from quaint rural landscapes to bustling urban metropolises. This narrative mirrors TNB’s trajectory, evolving from a traditional power provider to a pioneer championing sustainable energy solutions. It reflects the unwavering commitment to the Energy Transition Plan, with the ultimate aim of achieving Net Zero by 2050. As we celebrate our 75th anniversary, the cover serves as a testament to our enduring mission: Powering Growth, Catalysing Green. It showcases TNB’s leadership in fostering sustainable growth, spearheading the transition towards a greener future. The bottom illustration delves into the individual’s 

In [ ]:
df = pd.DataFrame(data_list)

In [ ]:
df

,text,page,embedding
0,75 75 Integrated Annual Report 2023 POWERING G...,"[1, 2]","[-0.0869140625, 0.007781982421875, -0.01696777..."
1,"aspirations for a Better, Brighter Future. Fro...","[2, 3]","[-0.06494140625, 0.0137939453125, -0.010437011..."
2,Corporate Structure 22 Our International Footp...,[3],"[-0.0654296875, -0.00138092041015625, -0.02465..."
3,Board Leadership & Effectiveness 147 Effective...,"[3, 4]","[-0.024169921875, 0.008544921875, -0.010803222..."
4,52 FROM OUR LEADERSHIP 02 NAVIGATION ICONS Thi...,[4],"[-0.0213623046875, 0.0294189453125, 0.00273132..."
...,...,...,...
853,Proxy 2 NRIC No./PassportNo.: or failing him/h...,[383],"[-0.052734375, -0.0223388671875, 0.00534057617..."
854,payment of the following Non-Executive Directo...,[383],"[-0.05126953125, -0.05859375, -0.023193359375,..."
855,Director from the date of his appointment unti...,"[383, 384]","[-0.051025390625, -0.0233154296875, -0.0280761..."
856,"members of the Company, in accordance with Sec...",[384],"[-0.0186767578125, -0.0034332275390625, -0.019..."


In [ ]:
df.to_parquet("data.parquet", engine="pyarrow")

In [ ]:
# RAG
# Embeddings
# Input from user
# Convert input into embedding
# Map input_embedding to the list of embeddings
# Return back the relevant documents

In [ ]:
input_str = "Apakah faktor yang meningkatkan keuntungan TNB?"

In [ ]:
payload = {
    "input": input_str,
    "model": "base"
}

In [ ]:
response = requests.post(url, json=payload, headers=headers)

json_response = response.json()

embedding = json_response["data"][0]["embedding"]

embedding_input = np.array(embedding)

In [ ]:
response.text

'{"object":"list","data":[{"object":"embedding","index":0,"embedding":[-0.0223388671875,0.003570556640625,-0.04052734375,0.03759765625,0.01043701171875,0.00616455078125,-0.017578125,0.0137939453125,0.0196533203125,0.039306640625,-0.0054931640625,0.0086669921875,-0.09375,0.01904296875,0.0084228515625,-0.04052734375,0.032470703125,-0.015869140625,-0.033203125,-0.0303955078125,-0.0228271484375,0.0001392364501953125,-0.041748046875,0.048095703125,-0.01031494140625,0.05908203125,-0.00836181640625,-0.02978515625,-0.000743865966796875,0.0537109375,-0.004180908203125,-0.0277099609375,-0.046875,0.0028076171875,-0.009765625,-0.017578125,0.041015625,-0.041015625,-0.0189208984375,-0.015625,0.0172119140625,0.029296875,-0.00072479248046875,-0.05517578125,0.036865234375,-0.0233154296875,0.0172119140625,-0.035400390625,-0.028564453125,-0.0172119140625,0.0179443359375,-0.03125,0.00604248046875,0.0263671875,0.0208740234375,0.037841796875,-0.056640625,-0.0001850128173828125,-0.01373291015625,-0.059082031

In [ ]:
json.loads(response.text)

{'object': 'list',
 'data': [{'object': 'embedding',
   'index': 0,
   'embedding': [-0.0223388671875,
    0.003570556640625,
    -0.04052734375,
    0.03759765625,
    0.01043701171875,
    0.00616455078125,
    -0.017578125,
    0.0137939453125,
    0.0196533203125,
    0.039306640625,
    -0.0054931640625,
    0.0086669921875,
    -0.09375,
    0.01904296875,
    0.0084228515625,
    -0.04052734375,
    0.032470703125,
    -0.015869140625,
    -0.033203125,
    -0.0303955078125,
    -0.0228271484375,
    0.0001392364501953125,
    -0.041748046875,
    0.048095703125,
    -0.01031494140625,
    0.05908203125,
    -0.00836181640625,
    -0.02978515625,
    -0.000743865966796875,
    0.0537109375,
    -0.004180908203125,
    -0.0277099609375,
    -0.046875,
    0.0028076171875,
    -0.009765625,
    -0.017578125,
    0.041015625,
    -0.041015625,
    -0.0189208984375,
    -0.015625,
    0.0172119140625,
    0.029296875,
    -0.00072479248046875,
    -0.05517578125,
    0.036865234375,

In [ ]:
input_embedding_base = json.loads(response.text)

In [ ]:
input_embedding_formatted = input_embedding_base["data"][0]["embedding"]

In [92]:
tools = [
    {
        "name": "get_food_recommendations",
        "description": "Receives details of food ",
        "parameters": {
            "type": "object",
            "properties": {
                "food": {
                    "type": "string",
                    "description": "The details of the food",
                },
            },
            "required": ["food"],
        },
    }
]

In [102]:
response = client.chat.completions.create(
    model="mallam-small",
    temperature=1.0,
    max_tokens=1024,
    messages=[
    {
        "role": "user",
        "content": 'kawan saya nazhan nak makan chicken noodle soup'
    }
],
    tools=tools
)
response = json.loads(response.choices[0].message.content)

In [103]:
food_input = response["arguments"]["food"]

In [104]:
food_input

'Chicken Noodle Soup'

In [106]:
api_url = 'https://api.api-ninjas.com/v1/recipe?query={}'.format(food_input)
response = requests.get(api_url, headers={'X-Api-Key': recipe_api})
if response.status_code == requests.codes.ok:
    pprint(response.text)
else:
    print("Error:", response.status_code, response.text)

('[{"title": "Soto Banjar Indonesian Chicken Noodle Soup", "ingredients": "4 c '
 'Defatted chicken broth|3 c Water|4 lg Flat-leaf parsley sprigs|2 Fresh '
 'cilantro sprigs; roots and stems slightly crushed|1 Whole chicken breast (1 '
 'to 1 1/4 lbs.); rinsed|3 md Boiling potatoes (about 12 oz.); peeled and cut '
 'into 1/2-inch cubes|4 oz Shallots; peeled|4 lg Cloves garlic; peeled|3 tb '
 'Olive oil|1 tb Peeled fresh ginger; minced|1 ts Ground cardamom|1 Cinnamon '
 'stick; (3 inches long)|Coarse salt and freshly ground black peppe; to '
 'taste|2 c Cooked angel hair pasta|3 Hard-cooked eggs; peeled and '
 'quartered|1/4 c Chopped green celery|Leaves or fresh cilantro leaves|1 Lime; '
 'thinly sliced", "servings": "1 Servings", "instructions": "Combine the '
 'chicken broth, water, and parsley and cilantro sprigs in a medium-size soup '
 'pot. Bring to a boil and add the chicken breast. Reduce the heat and simmer, '
 'partially covered, until the chicken is just cooked through, abou